In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
import requests
import time
import gmaps
from scipy.stats import linregress

# Import API key
from config import g_key

In [2]:
# Bring in datasets
mask_data_path = "Resources\mask-use-by-county.csv"
county_data_path = "Resources\lat_lon_county.csv"
death_data_path= "Resources\cleaned_death_by_county.csv"

# Read the mask data, death data, and county data
mask_data = pd.read_csv(mask_data_path)
county_data= pd.read_csv(county_data_path)
death_data= pd.read_csv(death_data_path)

In [3]:
# Rename columns
death_data= death_data.rename(columns={"countyFIPS": "County Code"})
county_data= county_data.rename(columns={"countyFIPS": "County Code"})
mask_data= mask_data.rename(columns={"COUNTYFP": "County Code"})

In [5]:
# Add columns for Never/Rarely/Sometimes (doesn't wear a mask) and Frequently/Always (usually wears a mask)
mask_data["Non-Compliance"]= mask_data["NEVER"] + mask_data["RARELY"] + mask_data["SOMETIMES"]
mask_data["Compliance"]= mask_data["FREQUENTLY"] + mask_data["ALWAYS"] 

# Set percentages as integers
mask_data["Non-Compliance"]= mask_data["Non-Compliance"]*100
mask_data["Compliance"]= mask_data["Compliance"]*100

# Delete old columns
del mask_data["NEVER"]
del mask_data["RARELY"]
del mask_data["SOMETIMES"]
del mask_data["FREQUENTLY"]
del mask_data["ALWAYS"]

mask_data.head()

,County Code,Non-Compliance,Compliance
0,1001,26.1,73.9
1,1003,24.0,75.9
2,1005,30.8,69.2
3,1007,15.0,85.0
4,1009,34.7,65.3


In [6]:
# Identify metro Atlanta counties for consideration
# Dekalb- 13089
# Fulton- 13121
# Cobb- 13067
# Gwinnett-13135
# Clayton- 13063
# Coweta- 13077
# Douglas-13097
# Fayette- 13113
# Henry-13151 

atlanta_counties= [13089, 13121, 13067, 13135, 13063, 13077, 13097, 13113, 13151]

In [7]:
# DataFrame of only metro Atlanta counties
atl_df= mask_data.loc[mask_data["County Code"].isin(atlanta_counties)]
atl_df

,County Code,Non-Compliance,Compliance
417,13063,21.7,78.3
419,13067,20.4,79.7
424,13077,34.1,65.9
430,13089,13.9,86.0
434,13097,21.0,78.9
442,13113,27.7,72.3
446,13121,15.1,84.9
453,13135,17.1,82.9
461,13151,24.2,75.9


In [12]:
# Merge to get deaths for Atlanta counties
atl_merge= pd.merge(atl_df, death_data, how= "left", on= ["County Code", "County Code"])
atl_deaths= atl_merge.dropna()

In [13]:
atl_deaths

,County Code,Non-Compliance,Compliance,County Name,State,County_State,Deaths as of October 19 2020,countyFIPS.1
0,13063,21.7,78.3,Clayton County,GA,"Clayton County, GA",174,13063
1,13067,20.4,79.7,Cobb County,GA,"Cobb County, GA",448,13067
2,13077,34.1,65.9,Coweta County,GA,"Coweta County, GA",60,13077
3,13089,13.9,86.0,DeKalb County,GA,"DeKalb County, GA",390,13089
4,13097,21.0,78.9,Douglas County,GA,"Douglas County, GA",71,13097
5,13113,27.7,72.3,Fayette County,GA,"Fayette County, GA",53,13113
6,13121,15.1,84.9,Fulton County,GA,"Fulton County, GA",607,13121
7,13135,17.1,82.9,Gwinnett County,GA,"Gwinnett County, GA",434,13135
8,13151,24.2,75.9,Henry County,GA,"Henry County, GA",111,13151


In [14]:
# Merge to get Lat/Lngs for these counties
atl_merge2= pd.merge(atl_deaths, county_data, how= "left", on= ["County_State", "County_State"])
atl_counties_deaths= atl_merge2.dropna()

In [15]:
atl_counties_deaths

,County Code_x,Non-Compliance,Compliance,County Name,State_x,County_State,Deaths as of October 19 2020,countyFIPS.1,State_y,County,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,Latitude,Longitude,County Code_y,g
0,13063,21.7,78.3,Clayton County,GA,"Clayton County, GA",174,13063,GA,Clayton County,3.669467e+08,6972579,141.679,2.692,33.542686,-84.355573,13063,13063
1,13067,20.4,79.7,Cobb County,GA,"Cobb County, GA",448,13067,GA,Cobb County,8.800524e+08,12376345,339.790,4.779,33.939930,-84.574112,13067,13067
2,13077,34.1,65.9,Coweta County,GA,"Coweta County, GA",60,13077,GA,Coweta County,1.142012e+09,12689156,440.933,4.899,33.352897,-84.762138,13077,13077
3,13089,13.9,86.0,DeKalb County,GA,"DeKalb County, GA",390,13089,GA,DeKalb County,6.934208e+08,8881087,267.731,3.429,33.770661,-84.226343,1049,1049
4,13097,21.0,78.9,Douglas County,GA,"Douglas County, GA",71,13097,GA,Douglas County,5.183550e+08,2423527,200.138,0.936,33.701242,-84.767314,8035,8035
5,13113,27.7,72.3,Fayette County,GA,"Fayette County, GA",53,13113,GA,Fayette County,5.034368e+08,12808490,194.378,4.945,33.412717,-84.493941,1057,1057
6,13121,15.1,84.9,Fulton County,GA,"Fulton County, GA",607,13121,GA,Fulton County,1.364208e+09,19715145,526.724,7.612,33.790034,-84.468182,5049,5049
7,13135,17.1,82.9,Gwinnett County,GA,"Gwinnett County, GA",434,13135,GA,Gwinnett County,1.115309e+09,15958551,430.623,6.162,33.959146,-84.023087,13135,13135
8,13151,24.2,75.9,Henry County,GA,"Henry County, GA",111,13151,GA,Henry County,8.252964e+08,20290464,318.649,7.834,33.452947,-84.154021,1067,1067


In [17]:
# Delete excess columns
del atl_counties_deaths["County Code_x"]
del atl_counties_deaths["State_x"]
del atl_counties_deaths["County"]
del atl_counties_deaths["ALAND"]
del atl_counties_deaths["AWATER"]
del atl_counties_deaths["ALAND_SQMI"]
del atl_counties_deaths["AWATER_SQMI"]
del atl_counties_deaths["County Code_y"]
del atl_counties_deaths["countyFIPS.1"]
del atl_counties_deaths["g"]
del atl_counties_deaths["State_y"]
del atl_counties_deaths["County_State"]

KeyError: 'County Code_x'

In [18]:
# Rename for easier variables
atl_counties_deaths= atl_counties_deaths.rename(columns= {"Deaths as of October 19 2020": "Deaths"})
atl_counties_deaths

,Non-Compliance,Compliance,County Name,County_State,Deaths,countyFIPS.1,State_y,Latitude,Longitude,County Code_y,g
0,21.7,78.3,Clayton County,"Clayton County, GA",174,13063,GA,33.542686,-84.355573,13063,13063
1,20.4,79.7,Cobb County,"Cobb County, GA",448,13067,GA,33.939930,-84.574112,13067,13067
2,34.1,65.9,Coweta County,"Coweta County, GA",60,13077,GA,33.352897,-84.762138,13077,13077
3,13.9,86.0,DeKalb County,"DeKalb County, GA",390,13089,GA,33.770661,-84.226343,1049,1049
4,21.0,78.9,Douglas County,"Douglas County, GA",71,13097,GA,33.701242,-84.767314,8035,8035
5,27.7,72.3,Fayette County,"Fayette County, GA",53,13113,GA,33.412717,-84.493941,1057,1057
6,15.1,84.9,Fulton County,"Fulton County, GA",607,13121,GA,33.790034,-84.468182,5049,5049
7,17.1,82.9,Gwinnett County,"Gwinnett County, GA",434,13135,GA,33.959146,-84.023087,13135,13135
8,24.2,75.9,Henry County,"Henry County, GA",111,13151,GA,33.452947,-84.154021,1067,1067


In [19]:
atl_map= atl_counties_deaths

In [21]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Use Lat and Lng as locations and humidity as weight 
locations = atl_map[["Latitude", "Longitude"]].astype(float)

In [22]:

# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>County</dt><dd>{County_State}</dd>
<dt>Deaths</dt><dd>{Deaths}</dd>
<dt>Mask Compliance (%)</dt><dd>{Compliance}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
atl_info = [info_box_template.format(**row) for index, row in atl_map.iterrows()]
locations = locations = atl_map[["Latitude", "Longitude"]]

In [24]:
# Plot markers to layer on top of heatmap
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=atl_info)

# Display figure
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))